# Generative adversarial Network (GAN )

### Mnist DataSet

In [3]:
from tensorflow import keras 

from keras.datasets import mnist
from keras import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,LeakyReLU,Dropout,Conv2DTranspose,Reshape


import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

## Discriminator


try classify images to real or fake 

In [4]:
def Discriminator(in_shape=(28,28,1)):
  model=Sequential()
  model.add(Conv2D(64,(3,3),activation='relu',input_shape=in_shape))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(64,(3,3)))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model 


## Generator 

generate new examples

In [5]:
def Genertor(latent_dim):
  model=Sequential()

  n_nodes=128*7*7
  model.add(Dense(n_nodes,input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((7,7,128)))
  #unsample to 14x14 
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  #unsample to 28x28
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(1,(7,7),activation='sigmoid',padding='same'))

  # image 28x28
  return model 

## Image plot 

In [6]:
def plott(examples,n=5):
  
  for i in range(n*n):
    plt.subplot(n,n,i+1)
    plt.axis('off')
    plt.imshow(exampes[i,:,:,0],cmap='gray_r')
    plt.show()

## Combine Gerator model and Dicriminator Model

In [7]:
def GAN(g_model,d_model):
  d_model.trainable=False

  model=Sequential()
  model.add(g_model)
  model.add(d_model)


  model.compile(optimizer='adam',loss='binary_crossentropy')
  return model 

## Load Data Set 

In [8]:
def load_real_axamples():
  (train_X,_),(_,_)=keras.datasets.mnist.load_data()
 #28,28
  X=np.expand_dims(train_X,axis=-1)   #28,28,1
  X=X.astype('float32')

  X=X/255.0

  return X

## select real example 


In [9]:
def generate_real_example(dataset,n_sanples):
  ix=np.random.randint(0,dataset.shape[0],n_sanples)

  X=dataset[ix]

  y=np.ones((n_sanples,1))  ## label ==> 1

  return X,y

generate points from latent space 

In [16]:
def generate_latent_space_point(laten_dim,n_samples):
  X_input =np.random.randn(latent_dim*n_samples)
  X_input=X_input.reshape(n_samples,laten_dim)

  return X_input

# use generator to generate fake example

In [17]:
def generate_fake_examples(g_model,latent_dim,n_samples):
  X_input=generate_latent_space_point(latent_dim,n_samples)

  X=g_model.predict(X_input)

  y=np.zeros((n_samples,1)) ## label --> 0

  return X,y


# Evaluate Discriminator 

In [18]:

def summarize_performance(g_model,d_model,datset,latent_dim,n_samples=100):
  X_real,y_real=generate_real_example(dataset,n_samples)

  _,acc_real=d_model.evaluate(X_real,y_real)


  X_fake,y_fake=generate_fake_examples(g_model,latent_dim,n_samples)

  _,acc__fake=d_model.evaluate(X_fake,y_fake)

  print("Accuarcy real {} % ,Fake {} %".format(acc_real*100,acc__fake*100))
  plott(X_fake,10)

## Train 

In [19]:
def train(g_model,d_model,GAN,dataset,latent_dim,n_epochs=100,n_batch=128):
  batch_per_epoch=int(dataset.shape[0]/n_batch)
  half_batch=int(n_batch/2)

  for i in range(n_epochs):

    for j in range(batch_per_epoch):
      #select random real image 
      X_real,y_real=generate_real_example(dataset,half_batch)

      #generate fake image 

      X_fake,y_fake=generate_fake_examples(g_model,latent_dim,half_batch)

      #create training set for discrimnitor 
      X,y=np.vstack((X_real,X_fake)),np.vstack((y_real,y_fake))  #X,y


      d_loss,_=d_model.train_on_batch(X,y)  #dicriminator


      ## Radom Sample
      X_gan=generate_latent_space_point(latent_dim,n_batch)

      y_gan=np.ones((n_batch,1))

      GAN_loss=GAN.train_on_batch(X_gan,y_gan)  


      print("Discriminator loss {} , GAN loss {} ".format(d_loss,GAN_loss))
  summarize_performance(g_model,d_model,datset,latent_dim)



  

In [ ]:
latent_dim=100
d_model=Discriminator()

g_model=Genertor(latent_dim)


Gan_model=GAN(g_model,d_model)

dataset=load_real_axamples()

train(g_model,d_model,Gan_model,dataset,latent_dim)

Streaming output truncated to the last 5000 lines.
Discriminator loss 0.4790356159210205 , GAN loss 1.2845274209976196 
Discriminator loss 0.446133553981781 , GAN loss 1.5136842727661133 
Discriminator loss 0.4347497224807739 , GAN loss 1.4250129461288452 
Discriminator loss 0.5217769742012024 , GAN loss 1.288097620010376 
Discriminator loss 0.4495875835418701 , GAN loss 1.2831610441207886 
Discriminator loss 0.47848784923553467 , GAN loss 1.3784215450286865 
Discriminator loss 0.4347188174724579 , GAN loss 1.4352515935897827 
Discriminator loss 0.552208662033081 , GAN loss 1.3034322261810303 
Discriminator loss 0.4506186842918396 , GAN loss 1.2143528461456299 
Discriminator loss 0.5331881046295166 , GAN loss 1.2477179765701294 
Discriminator loss 0.5283591747283936 , GAN loss 1.3002862930297852 
Discriminator loss 0.46244868636131287 , GAN loss 1.1167128086090088 
Discriminator loss 0.5204532146453857 , GAN loss 1.298757553100586 
Discriminator loss 0.47838401794433594 , GAN loss 1.39